In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#Change to the correct path if executed
%cd /content/gdrive/Shareddrives/NLI/Submission/Training

[Errno 2] No such file or directory: '/content/gdrive/Shareddrives/NLI/Submission/Training #Change to the path if executed'
/content/gdrive/Shareddrives/NLI/Submission/Training


In [ ]:
%cd User DA

/content/gdrive/Shareddrives/NLI/Submission/Training/User DA


In [ ]:
!pip install datasets

  Using cached datasets-2.15.0-py3-none-any.whl (521 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl (7.9 kB)
  Using cached dill-0.3.7-py3-none-any.whl (115 kB)
  Using cached multiprocess-0.70.15-py310-none-any.whl (134 kB)


In [ ]:
! pip install evaluate

  Using cached evaluate-0.4.1-py3-none-any.whl (84 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)


In [ ]:
! pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-zirqywn7
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-zirqywn7
  Resolved https://github.com/huggingface/transformers to commit e660424717daf47d4e511a78b9dda230a2f2a602
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8217653 sha256=34d88d0c3de122f2f8949c3f0d2122e3dad141fff371ff3a4a9b7cdce6e9af9c
  Stored in directory: /tmp/pip-ephem-wheel-cache-djvx1twd/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
! pip install accelerate -U

In [ ]:
!ls

data  Training_user_dialogAct.ipynb


In [ ]:
# --test_file shall be provided only along with --do_predict
! python ../run_classification.py --model_name_or_path  bert-large-uncased --train_file "data/user_dA_train.json" --validation_file "data/user_dA_validation.json" --test_file "data/user_dA_test.json" --shuffle_train_dataset --metric_name accuracy --text_column_name "input" --label_column_name "output" --do_train --do_eval --do_predict --max_seq_length 128 --per_device_train_batch_size 32 --learning_rate 2e-5 --num_train_epochs 1 --output_dir results_bert_large_uncased_user_da

`AnnotionFormat` is deprecated and will be removed in v4.38. Please use `transformers.image_utils.AnnotationFormat` instead.
2023-12-12 00:06:56.449771: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 00:06:56.449859: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 00:06:56.449920: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 00:06:59.603706: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
12/12/2023 00:07:05 - WARNING - __main__ - Process rank: 0, device: cpu, n_gpu: 0, distributed training: False, 1

In [ ]:
from sklearn.metrics import classification_report
import pandas as pd

df_test = pd.read_json("./data/user_dA_test.json")
df_test_preds = pd.read_csv("results_bert_large_uncased_user_da/predict_results.txt", sep="\t")
true_labels = df_test["output"].tolist()
pred_labels = df_test_preds["prediction"].tolist()
print(classification_report(true_labels, pred_labels))

In [ ]:
text = "I am also looking for The Riverside Brasserie. | Restaurant-Inform | riverside brasserie is a modern european restaurant in the centre. Its address is Doubletree by Hilton Cambridge Granta Place Mill Lane and postcode is cb21rt. Want to book it? | Booking-Inform, Restaurant-Inform | Yes, I would like to book it and get a reference number."
text = "Sure, when would you like that reservation? | Booking-Request | i want to book it for 2 people and 2 nights starting from saturday. | Hotel-Inform"
gt_output = 'Restaurant-Request'

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")
model = AutoModelForSequenceClassification.from_pretrained("T3_1_multilabeled_dataset_approach").to(device)
model_config = json.load(open("T3_1_multilabeled_dataset_approach/config.json", "r"))

encoding = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=128,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

print('device:', device)
input_ids = encoding["input_ids"].to(device)
attention_mask = encoding["attention_mask"].to(device)

with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1)

print(predicted_labels)
pred_id = str(predicted_labels.cpu().detach().numpy()[0])
print(model_config["id2label"][pred_id])

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

device: cuda
tensor([5], device='cuda:0')
Booking-Book


In [ ]:
# Apply sigmoid to convert logits to probabilities
probabilities = torch.sigmoid(logits)
# Define a threshold
threshold = 0.3
# Apply threshold to determine label assignments
predicted_labels = (probabilities > threshold).int()
non_zero_positions = [i for i, value in enumerate(predicted_labels[0]) if value != 0]
for i in non_zero_positions:
  print(model_config["id2label"][str(i)])

Booking-Book
general-reqmore
